<a href="https://colab.research.google.com/github/Lucia-Garcia-Lado/Graph-Coloring-Models-for-Production-Line-Scheduling-Optimization/blob/main/graph_coloring_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Coloring Models for Production Line Scheduling Optimization

> Lucía García Lado <br>
> Alejandro Sosa Corral

In [1]:
!pip install deap -q

In [2]:
from deap import base, creator, tools, algorithms
import random

In [3]:
# ------------------------------------------------------------
# 1. Carga de datos para dataset JSPLIB
# ------------------------------------------------------------

def load_instance(filename):
    operations = []

    with open(filename, 'r') as f:
        # Leemos las líneas, ignorando comentarios (#)
        lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]

    # Eliminamos el header y tomamos el número de trabajos y máquinas
    header = lines[0].split()
    num_jobs = int(header[0])
    num_machines = int(header[1])

    # El resto de las líneas son trabajos
    job_lines = lines[1:]

    if len(job_lines) < num_jobs:
        raise ValueError(f"File format error: Expected {num_jobs} job lines, found {len(job_lines)}")

    for job_id in range(num_jobs):
        line = job_lines[job_id]
        values = list(map(int, line.split()))

        # Emparejamos: (Máquina, Duración)
        order = 0
        for i in range(0, len(values), 2):
            machine = values[i]
            duration = values[i+1]

            operations.append({
                "job": job_id,
                "machine": machine,
                "duration": duration,
                "order": order
            })
            order += 1

    return operations, num_jobs, num_machines

In [4]:
# ------------------------------------------------------------
# 2. Problema: FT06 (6 Trabajos, 6 Máquinas)
# ------------------------------------------------------------
# Makespan Óptimo (conocido): 55

OPERATIONS, NUM_JOBS, NUM_MACHINES = load_instance("ft10")
NUM_OPS = len(OPERATIONS)

HORIZON = sum(op["duration"] for op in OPERATIONS)

print(f"Dataset loaded. Horizon set to: {HORIZON}")

# Variables genéticas
POPULATION_SIZE = max(NUM_OPS*15, 200)
NUM_GENERATIONS = NUM_OPS*20
CROSS_PROBABILITY = 0.8
MUT_PROBABILITY = 0.6

TOURNAMENT_SIZE = 2
ELITE_SIZE = 5

START_SHIFT = int(HORIZON*0.3)
END_SHIFT = 1

Dataset loaded. Horizon set to: 5109


In [5]:
# 1. Mapa ID Máquina -> Lista de (índice, operación)
MACHINE_MAP = {}
machine_ids = set(op["machine"] for op in OPERATIONS)
for m in machine_ids:
    MACHINE_MAP[m] = [(i, op) for i, op in enumerate(OPERATIONS) if op["machine"] == m]

# 2. Mapa ID Trabajo -> Lista de (índice, operación) ordenadas por secuencia
JOB_MAP = {}
job_ids = set(op["job"] for op in OPERATIONS)
for j in job_ids:
    ops = [(i, op) for i, op in enumerate(OPERATIONS) if op["job"] == j]
    ops.sort(key=lambda x: x[1]["order"])
    JOB_MAP[j] = ops

# ------------------------------------------------------------
# 3. Función de Fitness (Evaluación)
# ------------------------------------------------------------

def evaluate(ind):
    # --- CONFLICTOS DE MÁQUINA ---
    machine_conflicts = 0

    for machine_id, machine_ops in MACHINE_MAP.items():

        # Comparamos cada par de operaciones en la misma máquina
        for i in range(len(machine_ops)):
            for j in range(i + 1, len(machine_ops)):

                index_A, op_data_A = machine_ops[i]
                index_B, op_data_B = machine_ops[j]

                start_A = ind[index_A]
                end_A = start_A + op_data_A["duration"]

                start_B = ind[index_B]
                end_B = start_B + op_data_B["duration"]

                # Comprobamos el overlap
                if start_A < end_B and start_B < end_A:
                    machine_conflicts += 1

    # --- CONFLICTOS DE PRECEDENCIA ---
    precedence_conflicts = 0

    for job_id, job_steps in JOB_MAP.items():

        # Iteramos sobre los pasos secuenciales
        for k in range(len(job_steps) - 1):
            current_step_idx, current_step_data = job_steps[k]
            next_step_idx, next_step_data = job_steps[k+1]

            current_step_end_time = ind[current_step_idx] + current_step_data["duration"]
            next_step_start_time = ind[next_step_idx]

            # El siguiente paso no puede empezar antes de que termine el actual
            if next_step_start_time < current_step_end_time:
                precedence_conflicts += 1

    # --- OBJETIVOS ---
    end_times = [ind[i] + OPERATIONS[i]["duration"] for i in range(NUM_OPS)]
    Cmax = max(end_times) if end_times else 0

    # Energía (Peak Load)
    events = []
    for i in range(NUM_OPS):
        start = ind[i]
        end = start + OPERATIONS[i]["duration"]
        events.append((start, 1))   # Máquina ON
        events.append((end, -1))    # Máquina OFF

    events.sort()

    peak_load = 0
    current_load = 0
    for _, change in events:
        current_load += change
        if current_load > peak_load:
            peak_load = current_load

    energy = peak_load

    # --- SUMA PONDERADA ---
    alpha = 1000  # Penalización alta por conflicto de máquina
    beta = 1000   # Penalización alta por conflicto de precedencia
    gamma = 10    # Peso para Carga Pico
    delta = 0.1   # Peso para Makespan

    total_cost = (alpha * machine_conflicts) + \
                 (beta * precedence_conflicts) + \
                 (gamma * energy) + \
                 (delta * Cmax)

    return total_cost,

In [6]:
# ------------------------------------------------------------
# 4. Operadores Genéticos
# ------------------------------------------------------------

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Generación de individuos factibles
def generate_feasible_sequence():
    ind = [0] * NUM_OPS

    start_range = int(HORIZON * 0.3)
    if start_range < 1: start_range = 1

    slack_range = int(HORIZON * 0.15)
    if slack_range < 1: slack_range = 1

    for job_id, steps in JOB_MAP.items():

        current_time = random.randint(0, start_range)

        for idx, op_data in steps:
            # Asignamos el tiempo actual a este paso
            ind[idx] = current_time

            duration = op_data["duration"]

            # Calculamos cuándo debe empezar el SIGUIENTE paso (Fin del actual + un "Slack" aleatorio (hueco)).
            slack = random.randint(0, slack_range)
            current_time += duration + slack

            if current_time > HORIZON:
                current_time = HORIZON

    return creator.Individual(ind)

# Generación de población
toolbox.register("individual", generate_feasible_sequence)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operadores estándar
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)
toolbox.register("mate", tools.cxTwoPoint)

def multi_shift_mutation(ind, max_shift, indpb):  # Multi-shift nos permite explorar de manera más eficiente que el shift normal
    did_mutate = False
    for i in range(len(ind)):
        if random.random() < indpb:
            ind[i] += random.randint(-max_shift, max_shift)
            ind[i] = max(0, min(HORIZON, ind[i]))
            did_mutate = True

    if not did_mutate:
        idx = random.randrange(NUM_OPS)
        ind[idx] += random.randint(-max_shift, max_shift)
        # Mantener dentro de los límites válidos [0, HORIZON]
        ind[idx] = max(0, min(HORIZON, ind[idx]))
    return ind,

toolbox.register("mutate", multi_shift_mutation, max_shift=20, indpb=0.1)

In [7]:
# ------------------------------------------------------------
# 5. Bucle Principal del Algoritmo Genético
# ------------------------------------------------------------

def main():
    pop = toolbox.population(n=POPULATION_SIZE)

    # Evaluamos la población inicial
    for ind in pop:
        ind.fitness.values = toolbox.evaluate(ind)

    print(f"Iniciando evolución... (Horizonte={HORIZON}, Pob={POPULATION_SIZE})")

    for gen in range(NUM_GENERATIONS):

        # Calculamos progreso para shift variable
        progress = gen/NUM_GENERATIONS
        current_max_shift = int(START_SHIFT * (1 - progress) + END_SHIFT * progress)

        toolbox.register("mutate", multi_shift_mutation, max_shift=current_max_shift, indpb=0.1)

        # Generar descendencia
        offspring = algorithms.varAnd(pop, toolbox, CROSS_PROBABILITY, MUT_PROBABILITY)

        # Evaluar descendencia
        for ind in offspring:
            ind.fitness.values = toolbox.evaluate(ind)

        # ELITISMO: mantenemos los mejores padres
        elite = tools.selBest(pop, ELITE_SIZE)

        # Seleccionar el resto de la población desde la descendencia
        rest = toolbox.select(offspring, POPULATION_SIZE - ELITE_SIZE)

        # Nueva población = élite + seleccionados del offspring
        pop = elite + rest
        best = tools.selBest(pop, k=1)[0]

        # Imprimimos solo cada 10 generaciones para evitar llenar la consola
        if gen % 10 == 0:
            print(f"Gen {gen:3d} | Mejor coste: {best.fitness.values[0]:.2f} | Shift actual: {current_max_shift}")

    # Resultados Finales
    best_final = tools.selBest(pop, k=1)[0]
    print("\n--- Resultado Final ---")
    print(f"Mejor coste: {best_final.fitness.values[0]:.2f}")
    print(f"Horario (Start Times): {best_final}")

    cost_val = best_final.fitness.values[0]
    if cost_val < 1000:
        print("La solución es FACTIBLE")
    else:
        print("La solución es INFACTIBLE (Penalizaciones > 0)")

if __name__ == "__main__":
    main()

Iniciando evolución... (Horizonte=5109, Pob=1500)
Gen   0 | Mejor coste: 4558.80 | Shift actual: 1532
Gen  10 | Mejor coste: 4558.80 | Shift actual: 1524
Gen  20 | Mejor coste: 4558.80 | Shift actual: 1516
Gen  30 | Mejor coste: 4558.80 | Shift actual: 1509
Gen  40 | Mejor coste: 4558.80 | Shift actual: 1501
Gen  50 | Mejor coste: 3558.80 | Shift actual: 1493
Gen  60 | Mejor coste: 3558.80 | Shift actual: 1486
Gen  70 | Mejor coste: 3558.80 | Shift actual: 1478
Gen  80 | Mejor coste: 3558.80 | Shift actual: 1470
Gen  90 | Mejor coste: 3558.80 | Shift actual: 1463
Gen 100 | Mejor coste: 3558.80 | Shift actual: 1455
Gen 110 | Mejor coste: 3558.80 | Shift actual: 1447
Gen 120 | Mejor coste: 3558.80 | Shift actual: 1440
Gen 130 | Mejor coste: 3558.80 | Shift actual: 1432
Gen 140 | Mejor coste: 3558.80 | Shift actual: 1424
Gen 150 | Mejor coste: 3558.80 | Shift actual: 1417
Gen 160 | Mejor coste: 2558.80 | Shift actual: 1409
Gen 170 | Mejor coste: 2558.80 | Shift actual: 1401
Gen 180 | Mejo